# AQA-II Lab 11 Models for Panel Data

### Eric G. Zhou, NYU Wagner/Med Scool
### 04/16/2020

## Panel data

Panel data is different from cross-sectional data in being more complex in the data structure. Most of the previous methods we learned are illustrated with cross-sectional data. However, with the development of new research interests and the way we collect data, panel data along with methods suitable for analyzing them emerge in recent decades. They become important tools for us to understand and study variations over time and across individuals/groups.

- A linear regression model with **cross-sectional** data would look like this:

$$ Y_i = \alpha + \beta_1 X_{i,1} + \beta_2 X_{i,2} + ... + \beta_k X_{i,K} + \epsilon_i, $$
or equivalently
$$ Y_i = \alpha + \beta_{k} X_{i,k} + \epsilon_i, $$

where the dependent variable $Y$ and error term $\epsilon$ has the dimension of $[N]$, which is the total number of the observation in this dataset, whereas the independent variable $X$ has the dimension of $[N,K]$, where $K$ is the number of different independent variables.

- A linear regression model with **panel data** would look like this:

$$ Y_{it} = \alpha + \beta_1 X_{it,1} + \beta_2 X_{it,2} + ... + \beta_k X_{it,k} + \epsilon_{it}, $$
or equivalently
$$ Y_{it} = \alpha + \beta_k X_{itk} + \epsilon_{it}.$$

What's new compared to the cross-secitonal model is that we for each invidiual or unit we have repeated measurment over time, indicated by the subscripted $t$. No matter the data is formatted as wide or long, we have another dimension in panel data. 

A couple of more definitions:
- Balanced panel: each individual/unit in the dataset is observed the same number of times, usually denoted T
- Unbalanced panel: individuals/units may be observed different number of times
- fixed panel: the same set of individuals is observed for the duration of the study

Previously, we learned about missing/attrition, which is a main reason of unbalanced panel. Most likely the panel dataset we will be working with
is an unbalanced panel.


## Models for Panel Data

### Quick motivation

for panel data methods, imgaine for the world we live, eveverything is changing all the time, the `trajectory` of person A and B `over time` might be different for many reasons. However, the variation overtime for person A and B might be `stable within oneself` (this might not be true always, so we will add flexibilities in modeling this too later on). Besides, we might not be able to measure all these factors affecting one' trajectory. What we could do though is to break down the error into two parts, a individual/unit-specific error term $u_i$, and a common error term $w_{it}$. And importantly, $\epsilon_{it} = u_i + w_{it}$.

Suppose we could measure the same variable for A and B overtime for a total of $T$ times, the general model we described above can now be written as :

\begin{align}
Y_{it} & = \alpha + \beta_k X_{itk} + \epsilon_{it} \\
& = \alpha + \beta_k X_{itk} + u_i + w_{it}
\end{align}

Sometimes, people would want to group $\alpha$ with $u_i$, so the equation becomes:

$$Y_{it} = \beta_k X_{itk} + \underbrace{c_i}_{= u_i + \alpha_i} + w_{it},$$

hence here $c_i$ is often referred to as individual heterogenous effect.

Given the richer information overtime from the panel data as opposed to cross-sectional data, also a special data structure, we could better use methods specially designed for these new tasks.

### Common assumptions

Similar to the OLS estimates, we will also need some basic assumptions that are needed across different types of methods:

1. Linearity
2. Full rank 
3. Exogeneity of the independent variables
4. Homoscedasticity and nonautocorrelation
5. Data generating mechanism - independent observations

### A summary for the models we will introduce today

1. **Pooled OLS**: a straightforward method that fails to accomodate the special panel data structure
2. **Fixed effect model**: also known as within-variation estimator, which makes assumptions to accomodate the special structure
3. **Random effects model**: makes strong assumptions to accomodate the special structure, and is a weighted average of the between- and within- effects; can be estimated via Generalized Least Squares (GLS) or MLE
4. **Random parameters**: also known as **hierarchical model**, a very flexible model allowing for variations in slopes over-time, based on the random effects model.



### Introducing the data we will use today

In [12]:
use http://www.stata-press.com/data/r13/nlswork4
sort idcode year
describe


(National Longitudinal Survey.  Young Women 14-26 years of age in 1968)



Contains data from http://www.stata-press.com/data/r13/nlswork4.dta
  obs:        28,534                          National Longitudinal Survey.
                                                Young Women 14-26 years of age
                                                in 1968
 vars:             6                          29 Jan 2013 16:35
--------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
--------------------------------------------------------------------------------
idcode          int     %8.0g                 NLS ID
year            byte    %8.0g                 interview year
age             byte    %8.0g                 age in current year
msp             byte    %8.0g                 1 if married, spouse present
ttl_exp         float   %9.0g                 total work exper

- Let's see if the data has a panel structure

In [11]:
%head 20

,idcode,year,age,msp,ttl_exp,ln_wage
1,1,70,18,0,1.0833333,1.451214
2,1,71,19,1,1.275641,1.0286198
3,1,72,20,1,2.2564101,1.5899774
4,1,73,21,1,2.3141024,1.7802728
5,1,75,23,1,2.775641,1.7770116
6,1,77,25,0,3.775641,1.7786806
7,1,78,26,0,3.8525641,2.4939759
8,1,80,28,0,5.2948718,2.5517154
9,1,83,31,0,5.2948718,2.4202614
10,1,85,33,0,7.1602564,2.6141725


- in this dataset, `idcode` is the $i$ dimension, `year` is the $t$ dimension, we have 3 independent variables `age`, `msp` and `tt_exp`, and 1 dependent variable `ln_wage`. Conincidentlly, all 3 independent variables are time-variant.

In [13]:
xtset idcode year

       panel variable:  idcode (unbalanced)
        time variable:  year, 68 to 88, but with gaps
                delta:  1 unit


- the command above is to set up the data as a panel structure that `stata` understands. As you can see, it's not a balanced panel, i.e., some respondents attrited in some years. 

In [14]:
duplicates report idcode year


Duplicates in terms of idcode year

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |        28534             0
--------------------------------------


- in dataset like this, it's always to check for duplicates -- you don't want to doule count your estimates. Results above show that we do not have duplicates in terms of `idcode` and `year`.

- Question for you, why do we check duplicates for `idcode` and `year`? Should we check for other variables? Why or why not?

## 1. The pooled regression model

The simplest model and often a starting point of panel data analysis. The specification goes as:

$$ y_{it} = \alpha + x^\prime_{it} \beta + \epsilon_{it}$$

For us to unbiasedly estimate $\beta$, we need the assumptions below:



\begin{align}
E[\epsilon_{it}|X_{it}] & = 0 \\
Var[\epsilon_{it}|X_{it}] & = \sigma^2_{\epsilon} \\
Cov[\epsilon_{it},\epsilon_{js}|X_{it}] & = 0 \text{ if } i \neq j \text{ or } t \neq s
\end{align}

Question, what does the 3rd assumption talk about? 

However, in reality, the 3rd assumption is almost never met. So we would have a problem of **autocorrelation**. 

As from the specification under motivation section, we can see that $E[w_{it}w_{is}] = \sigma^2_{u}$ when $t \neq s$. This actially says the unexplained error term is correlated when the same unit i is measured at different time points. 

The consequences of this problem is that the OLS estimator may be consistent, but the conventional variance estimator will be an underestimate of true variance. In other words, $\hat{\beta}_{ols} \sim = \beta$, and $Var(\hat{\beta})\leq Var(\beta)$. We have two remedies, one is your old friend, **robust covariance matrix estimation** and another is **cluster and stratification**

Below is an exmaple of how to implement this in `stata`:

In [18]:
quietly reg ln_wage age msp ttl_exp
estimates store m1
quietly reg ln_wage age msp ttl_exp, r
estimates store m2
quietly reg ln_wage age msp ttl_exp, r cluster(id)
estimates store m3

In [20]:
estimate table m1 m2 m3, stats(N r2_a) b(%7.2f) se(%7.3f) ti("Results of pooled OLS")


Results of pooled OLS

--------------------------------------------
    Variable |   m1        m2        m3     
-------------+------------------------------
         age |   -0.01     -0.01     -0.01  
             |   0.001     0.001     0.001  
         msp |    0.01      0.01      0.01  
             |   0.005     0.005     0.010  
     ttl_exp |    0.05      0.05      0.05  
             |   0.001     0.001     0.002  
       _cons |    1.55      1.55      1.55  
             |   0.014     0.014     0.024  
-------------+------------------------------
           N |   28494     28494     28494  
        r2_a |    0.18      0.18      0.18  
--------------------------------------------
                                legend: b/se


### The between- and within-groups estimators

To make things slightly more complicated, let's see how our estimations can be different. 

We talked about this: 
$$ y_{it} = \alpha + x^\prime_{it} \beta + \epsilon_{it}$$

We can also estimate in terms of group means:

$$\bar{y}_{i} = \alpha + \bar{x}^\prime_{i} \beta + \bar{\epsilon}_{i}$$

also in terms of deviations from the group means:
$$y_{it} - \bar{y}_{i} = \alpha + (x_{it} - \bar{x}_{i})^\prime \beta + \epsilon_{it} - \bar{\epsilon}_{i}$$

lastly, let's denote $\bar{\bar{x}}$ and $\bar{\bar{y}}$ as the overall mean for the dataset. 

Remember from AQA-I, we learned the notion of total sums of squares, and it's important to know the following:

$$ S^{total}_{xy} = \sum^n_{i=1} \sum^T_{t=1} (x_{it} - \bar{\bar{x}}) (y_{it} - \bar{\bar{y}})$$

so the within-group sums of squares are: 
$$ S^{within}_{xy} = \sum^n_{i=1} \sum^T_{t=1} (x_{it} - \bar{x_i}) (y_{it} - \bar{y_i})$$

so the between-group sums of squares are: 
$$ S^{between}_{xy} = \sum^n_{i=1} \sum^T_{t=1} (\bar{x}_{i} - \bar{\bar{x}}) (\bar{y_{i}} - \bar{\bar{y}})$$

it's not hard to see that 

$$ S^{total}_{xy} =  S^{within}_{xy} + S^{between}_{xy} $$

similiarly we have the properties for 

$$ S^{total}_{xx} =  S^{within}_{xx} + S^{between}_{xx} $$

Mostly importantly and equivalent to the OLS estimator,

$$b^{total} = [S^{total}_{xx}]^{-1} S^{total}_{xy} = [S^{within}_{xx} + S^{between}_{xx}]^{-1} [S^{within}_{xy} + S^{between}_{xy}]$$

Based on the equations above, we can see that 

$$ b^{within} = [S^{within}_{xx}]^{-1}S^{within}_{xy} $$

and 

$$ b^{between} = [S^{between}_{xx}]^{-1}S^{between}_{xy} $$

Question for you, what can you say about $b^{total}$? 

## 2. Fixed effects model

The fixed effects model aries from the assumption that the **omitted effects $c_i$** in the general model is **correlated** with the included variables.

$$ Y_{it} = \beta_k X_{itk} + \underbrace{c_i}_{= u_i + \alpha_i} + w_{it},$$

To control for the individual-specific effects that does not vary across time, we are given each individual a separate intercept. Because this specification has the same effect has adding a dummy variable for each person, the model is sometimes referred to as the **least squares dummy variable model (LSDV)**.

We could write down the estimator 

$$ b = [X^\prime M_D X]^{-1} [X^\prime M_D Y] = b^{within}$$

### Two-way fixed effects

People are abmitious and wish to not only control for inidividual-specific unobserables, but also time-specifics. So people invent this method, which turns out to be a natural extension of the usual one-way fixed effects model. 

$$ Y_{it} = \beta_k X_{itk} + \delta_t + \underbrace{c_i}_{= u_i + \alpha_i} + w_{it}$$ 

Let's see how we can go at them in `stata`

In [33]:
quietly xtreg ln_wage age msp ttl_exp, fe
estimates store m4
quietly areg ln_wage age msp ttl_exp i.year, a(idcode)
estimates store m5

In [24]:
estimate table m4 m5, stats(N r2_a) b(%7.2f) se(%7.3f) ti("Compare LSDV & 2-way FE")


Compare LSDV & 2-way FE

----------------------------------
    Variable |   m4        m5     
-------------+--------------------
         age |   -0.01      0.01  
             |   0.001     0.010  
         msp |    0.00     -0.00  
             |   0.005     0.006  
     ttl_exp |    0.04      0.04  
             |   0.001     0.001  
             |
        year |
         69  |              0.06  
             |             0.016  
         70  |             -0.00  
             |             0.023  
         71  |              0.01  
             |             0.032  
         72  |             -0.01  
             |             0.042  
         73  |             -0.03  
             |             0.051  
         75  |             -0.07  
             |             0.071  
         77  |             -0.07  
             |             0.090  
         78  |             -0.08  
             |             0.101  
         80  |             -0.15  
             |             0.120  

## 3. Random effects

Compared to fixed effects, random effects model assumes that the unobserved individual-specific effect $c_i$ is not correlated with the included variables. The benefits of random effects is that we can greatly reduce the parameters to be estimated, but could result in inconsistent estimates, if the assumption is violated. 

The model writes as:

$$ Y_{it} = \beta_k X_{itk} +  (u_i + \alpha) + w_{it}$$

and we say the $u_i$ is the random heteogeneity specific to the ith observation and is constant through time.

Suppose we could write the sum of the two error terms into one as:

$$ \eta_{it} = w_{it} + u_i$$

The error components can be broken down into 

$$E[\eta^2_{it}|X] = \sigma^2_{w} + \sigma^2_{u}$$ 

and 

$$ E[\eta_{it} \eta_{is}|X] = \sigma^2_{u}, t \neq s$$ 

$$ E[\eta_{it} \eta_{js}|X] = 0, \forall \text{ } t \text{ and } s \text{ if } i \neq j$$ 


Here is how we can do this in `stata`

In [25]:
quietly xtreg ln_wage age msp ttl_exp, re
estimates store m6
quietly xtreg ln_wage age msp ttl_exp, mle
estimates store m7

In [26]:
estimate table m6 m7, stats(N r2_a) b(%7.2f) se(%7.3f) ti("Compare RE in GLS and MLE")


Compare RE in GLS and MLE

----------------------------------
    Variable |   m6        m7     
-------------+--------------------
_            |
         age |   -0.01            
             |   0.001            
         msp |    0.00            
             |   0.005            
     ttl_exp |    0.04            
             |   0.001            
       _cons |    1.61            
             |   0.016            
-------------+--------------------
ln_wage      |
         age |             -0.01  
             |             0.001  
         msp |              0.00  
             |             0.005  
     ttl_exp |              0.04  
             |             0.001  
       _cons |              1.61  
             |             0.016  
-------------+--------------------
sigma_u      |
       _cons |              0.33  
             |             0.004  
-------------+--------------------
sigma_e      |
       _cons |              0.30  
             |             0.001  
--

In [27]:
xtreg ln_wage age msp ttl_exp, re


Random-effects GLS regression                   Number of obs     =     28,494
Group variable: idcode                          Number of groups  =      4,710

R-sq:                                           Obs per group:
     within  = 0.1373                                         min =          1
     between = 0.2552                                         avg =        6.0
     overall = 0.1797                                         max =         15

                                                Wald chi2(3)      =    5100.33
corr(u_i, X)   = 0 (assumed)                    Prob > chi2       =     0.0000

------------------------------------------------------------------------------
     ln_wage |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         age |  -.0069749   .0006882   -10.13   0.000    -.0083238   -.0056259
         msp |   .0046594   .0051012     0.91   0.361    -.00533

In [29]:
estimate table m3 m4 m5 m6 m7, stats(N r2_a) b(%7.2f) se(%7.3f) ti("Compare all models at once")


Compare all models at once

----------------------------------------------------------------
    Variable |   m3        m4        m5        m6        m7     
-------------+--------------------------------------------------
_            |
         age |   -0.01     -0.01      0.01     -0.01            
             |   0.001     0.001     0.010     0.001            
         msp |    0.01      0.00     -0.00      0.00            
             |   0.010     0.005     0.006     0.005            
     ttl_exp |    0.05      0.04      0.04      0.04            
             |   0.002     0.001     0.001     0.001            
             |
        year |
         69  |                        0.06                      
             |                       0.016                      
         70  |                       -0.00                      
             |                       0.023                      
         71  |                        0.01                      
             |  

At the very last, how do we know whether to use FE or RE? 

- assumptions
- theories
- strategies
- also tests

In [31]:
quietly xtreg ln_wage age msp ttl_exp, re
hausman m4 ., sigmamore




                 ---- Coefficients ----
             |      (b)          (B)            (b-B)     sqrt(diag(V_b-V_B))
             |       m4           .          Difference          S.E.
-------------+----------------------------------------------------------------
         age |    -.005485    -.0069749        .0014899        .0004803
         msp |    .0033427     .0046594       -.0013167        .0020596
     ttl_exp |    .0383604     .0429635       -.0046031        .0007181
------------------------------------------------------------------------------
                           b = consistent under Ho and Ha; obtained from xtreg
            B = inconsistent under Ha, efficient under Ho; obtained from xtreg

    Test:  Ho:  difference in coefficients not systematic

                  chi2(3) = (b-B)'[(V_b-V_B)^(-1)](b-B)
                          =      260.40
                Prob>chi2 =      0.0000


Results above shows that RE model is rejected compared to FE -- which makes sense as we have only 3 variables in the regression equation, a lot variation in the data is left unexplained. 